# Accessing and Visualizing HyTES data

## BioSCape Data Skills Workshop: From the Field to the Image
![Bioscape](images/121229-87.png)

[**BioSCape**](https://www.bioscape.io/), the Biodiversity Survey of the Cape, is NASA’s first biodiversity-focused airborne and field campaign that was conducted in South Africa in 2023. BioSCape’s primary objective is to study the structure, function, and composition of the region’s ecosystems, and how and why they are changing. 

BioSCape's airborne dataset is unprecedented, with `AVIRIS-NG`, `PRISM`, and `HyTES` imaging spectrometers capturing spectral data across the UV, visible and infrared at high resolution and `LVIS` acquiring coincident full-waveform lidar. BioSCape's `field dataset` is equally impressive, with 18 PI-led projects collecting data ranging from the diversity and phylogeny of plants, kelp and phytoplankton, eDNA, landscape acoustics, plant traits, blue carbon accounting, and more

This workshop will equip participants with the skills to find, subset, and visualize the various BioSCape field and airborne (imaging spectroscopy and full-waveform lidar) data sets. Participants will learn data skills through worked examples in terrestrial and aquatic ecosystems, including: wrangling lidar data, performing band math calculations, calculating spectral diversity metrics, machine learning and image classification, and mapping functional traits using partial least squares regression. The workshop format is a mix of expert talks and interactive coding notebooks and will be run through the BioSCape Cloud computing environment.

**Date:** October 9 - 11, 2024 Cape Town, South Africa</center>

**Host:** NASA’s Oak Ridge National Laboratory Distributed Active Archive Center (ORNL DAAC), in close collaboration with BioSCape, the South African Environmental Observation Network (SAEON), the University of Wisconsin Madison (Phil Townsend), The Nature Conservancy (Glenn Moncrieff), the University of California Merced (Erin Hestir), the University of Cape Town (Jasper Slingsby), Jet Propulsion Laboratory (Kerry Cawse-Nicholson), and UNESCO.

**Instructors:** 
- In-person contributors: Anabelle Cardoso, Erin Hestir, Phil Townsend, Henry Frye, Glenn Moncrieff, Jasper Slingsby, Michele Thornton, Rupesh Shrestha
- Virtual contributors: Kerry Cawse-Nicholson, Nico Stork, Kyle Kovach

**Audience:** This training is primarily intended for government natural resource management agency representatives and field technicians in South Africa, as well as local academics and students, especially those connected to the BioSCape Team. 


## Overview 
In this tutorial, we will access and visualize the airborne Hyperspectral Thermal Emission Spectrometer (HyTES) data for a flight line
- data were ordered from the [**NASA JPL HyTES website**](https://hytes.jpl.nasa.gov/order).

The Hyperspectral Thermal Emission Spectrometer (HyTES) is an airborne imaging spectrometer with 256 spectral channels between 7.5 and 12 micrometers in the thermal infrared (TIR) part of the electromagnetic spectrum and 512 pixels cross-track.

- HyTES provides high spatial and high spectral resolution data on **`surface temperature`** and **`emissivity`**
- HyTES acquires data in the thermal infrared (TIR)
    - TIR data are used to measure land surface temperature (LST), which informs models of water flux from land surface through processes such as evapotranspiration 

### Load Python Modules

In [ ]:
import xarray as xr
import hvplot.xarray
from os import path
import geopandas as gpd
from glob import glob

## HyTES Flights
A GeoJSON of the HyTES flight lines for the BioSCape project is available at the `shared` directory. Let's plot the Hytes dataset here.

In [ ]:
hytes_dir = "/shared/users/bioscape_ZA24workshop_data/RS_data/HyTES/"
hytes_flight = "HyTES.json"
hytes_gdf = gpd.read_file(path.join(hytes_dir, hytes_flight))
hytes_gdf.explore()

## HyTES Download
Currently, selected HyTES flights are available for download from the [JPL HyTES Website](https://hytes.jpl.nasa.gov/order).

We have placed an order and downloaded the HyTES data for a flight line at Table Mountain Box1ZA for 2023-11-13. They are at the `shared` folder as shown below:

In [ ]:
hytes_orderid = "20231113t102619_TableMountainBox1ZA"
hytes_dir = path.join(hytes_dir, hytes_orderid)
hytes_dir

In [ ]:
# find all hytes L1/L2 files
hytes_f = sorted(glob(path.join(hytes_dir, "*.hdf5")))
# print
for f in hytes_f:
    print(path.basename(f))

As we see above, there are two levels of HyTES products currently available for download.
- **Level 1 (L1)** product provides calibrated HyTES data in radiance units of W/m^2/µm/sr. The product also contains locational metadata from the instruments NGDCS, and per-pixel geolocation information, namely latitude, longitude, height, and number of steps taken during ray-casting
- **Level 2 (L2)** product contains following data:
    - Emissivity (L2_Emissivity: HyTES emissivity spectral data from 8-11.5 µm
    - PC Regression Emissivity  (L2_Emissivity_PC): HyTES emissivity data from 7.4-12 µm. L2_Emissivity_PC uses a Principal Component (PC) eigenvector regression approach to produce emissivity for all HyTES channels.
    - Land Surface Temperature (L2_LST): HyTES Land Surface Temperature (LST) data in units of Kelvin. Derived from atmospherically corrected level-1 radiance data using the TES algorithm.

### HyTES L1 product
Let's open the Level 1 file and plot the `radiance` band.

In [ ]:
hytes_l1 = [f for f in hytes_f if "_L1" in f][0]
# open dataset
ds_l1 = xr.open_dataset(path.join(hytes_dir, hytes_l1), engine="h5netcdf", phony_dims='access')
# renaming dimensions to sensible names
ds_l1 = ds_l1.rename({'phony_dim_0': 'y','phony_dim_1': 'x', 'phony_dim_2': 'bands'})
ds_l1

### Plotting L1 Radiance
Let's plot one of the radiance bands.

In [ ]:
ds_l1.sel(bands=60).radiance_data.hvplot.image(width=300, height=600, cmap='viridis',
                                              clim = (5, 18)).opts(invert_yaxis=True)

### HyTES L2 product
Let's open the L2 file.

In [ ]:
hytes_l2 = [f for f in hytes_f if "_L2" in f][0]
# open dataset
ds_l2 = xr.open_dataset(path.join(hytes_dir, hytes_l2), engine="h5netcdf", phony_dims='access')
# renaming dimensions to sensible names
ds_l2 = ds_l2.rename({'phony_dim_0': 'bands','phony_dim_1': 'y', 
                      'phony_dim_2': 'x', 'phony_dim_3': 'bands_emis'})
ds_l2

### Plotting L2 LST
Plot the variables `Land Surface Temperature (LST)`.

In [ ]:
ds_l2.L2_LST.hvplot.image(width=300, height=600,cmap='seismic',
             clim = (250, 350)).opts(invert_yaxis=True)